In [8]:
import json
import requests
import sys
from datetime import datetime, time

sys.path.insert(0, '../../../commons/')
sys.path.insert(0, '../../../virasana/')


from virasana.integracao.due import raspa_due

VIRASANA_URL = "http://localhost/virasana/"
VIRASANA_URL = "https://10.68.64.12/virasana/"

## Realizando consulta no MongoDB Gridfs do AJNA via API virasana

In [46]:
diaapesquisar = datetime.today()
diaapesquisar = datetime(2019, 11, 5)

In [47]:
datainicial = datetime.strftime(datetime.combine(diaapesquisar, time.min), '%Y-%m-%d  %H:%M:%S')
datafinal = datetime.strftime(datetime.combine(diaapesquisar, time.max), '%Y-%m-%d %H:%M:%S')
print(datainicial, datafinal)

2019-11-05  00:00:00 2019-11-05 23:59:59


In [48]:
params = {'query': 
          { 'metadata.dataescaneamento': {'$gte': datainicial, '$lte': datafinal},
            'metadata.contentType': 'image/jpeg',
            'metadata.carga.manifesto.tipomanifesto': 'lce'
          },
          'projection': 
          {'metadata.numeroinformado': 1,
           'metadata.dataescaneamento': 1}
         }

r = requests.post(VIRASANA_URL + "grid_data", json=params, verify=False)
lista_lce = list(r.json())
print(len(lista_lce))

params['query']['metadata.carga.manifesto.tipomanifesto'] = None
r = requests.post(VIRASANA_URL + "grid_data", json=params, verify=False)
lista_null = list(r.json())
print(len(lista_null))

# print(r.url)
# print(r.text)

c:\users\25052288840\pycharmprojects\ajna_docs\virasana\venv\lib\site-packages\urllib3\connectionpool.py:851: InsecureRequestWarning:

Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings

c:\users\25052288840\pycharmprojects\ajna_docs\virasana\venv\lib\site-packages\urllib3\connectionpool.py:851: InsecureRequestWarning:

Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings



29
2029


In [49]:
lista_lce

[{'_id': '5dc185e138d215300bffd9c9',
  'metadata': {'dataescaneamento': '2019-11-05 00:18:29',
   'numeroinformado': 'MNBU0098380'}},
 {'_id': '5dc185e038d215300bffd961',
  'metadata': {'dataescaneamento': '2019-11-05 00:28:01',
   'numeroinformado': 'MRKU7981144'}},
 {'_id': '5dc1860838d215300bffe1e1',
  'metadata': {'dataescaneamento': '2019-11-05 01:38:36',
   'numeroinformado': 'FDCU0597426'}},
 {'_id': '5dc185ff38d215300bffe146',
  'metadata': {'dataescaneamento': '2019-11-05 03:46:20',
   'numeroinformado': 'MNBU3245947'}},
 {'_id': '5dc1860938d215300bffe24b',
  'metadata': {'dataescaneamento': '2019-11-05 04:14:07',
   'numeroinformado': 'TRLU1713071'}},
 {'_id': '5dc1861638d215300bffe685',
  'metadata': {'dataescaneamento': '2019-11-05 06:08:33',
   'numeroinformado': 'HASU4371557'}},
 {'_id': '5dc1861638d215300bffe69d',
  'metadata': {'dataescaneamento': '2019-11-05 06:09:22',
   'numeroinformado': 'MRKU4336738'}},
 {'_id': '5dc1861638d215300bffe65d',
  'metadata': {'dataescan

In [50]:
# lista = [['MSCU6656780', '']]
lista = [*lista_lce, *lista_null]
conteineres_ids = {linha['metadata']['numeroinformado']: linha['_id'] for linha in lista}
conteineres = list(conteineres_ids.keys())

print(conteineres_ids)


{'MNBU0098380': '5dc185e138d215300bffd9c9', 'MRKU7981144': '5dc185e038d215300bffd961', 'FDCU0597426': '5dc1860838d215300bffe1e1', 'MNBU3245947': '5dc185ff38d215300bffe146', 'TRLU1713071': '5dc1860938d215300bffe24b', 'HASU4371557': '5dc1861638d215300bffe685', 'MRKU4336738': '5dc1861638d215300bffe69d', 'CAIU4903402': '5dc1861638d215300bffe65d', 'MSKU0988909': '5dc1861f38d215300bffe7d4', 'MRKU4513031': '5dc1862038d215300bffe904', 'MNBU0310458': '5dc1862238d215300bffea8b', 'HASU5128433': '5dc1862238d215300bffea83', 'MSKU1042264': '5dc1862a38d215300bffeb5b', 'MRKU4856447': '5dc1862c38d215300bffec8c', 'MSKU0372317': '5dc1862c38d215300bffec98', 'HLXU3477447': '5dc1862c38d215300bffece0', 'MSKU1769839': '5dc1863538d215300bffee29', 'MRKU4210143': '5dc1863438d215300bffee21', 'MSKU9260684': '5dc1863538d215300bffee2d', 'TTNU8217496': '5dc1863d38d215300bfff06f', 'TRIU8044800': '5dc1864738d215300bfff172', 'MNBU9017029': '5dc1864e38d215300bfff332', 'GESU5840127': '5dc31b8089f97bed10c671f2', 'MIEU30611

## Conectar na API do Pucomex/suiterfb para encontrar e baixar DUE 

In [51]:
# %%time
from selenium import webdriver

GECKO_PATH = 'C:\\Users\\25052288840\\Downloads\\chromedriver.exe'
driver = webdriver.Chrome(GECKO_PATH)
raspa_due.auth_suite_rfb(driver)
try:
    %time conteineres_listadue = raspa_due.get_dues_pos_acd(driver, conteineres)
    # print(conteineres_listadue)
    %time due_detalhe = raspa_due.detalha_dues(driver, conteineres_listadue)
    # print(conteineres_due)
finally:
    driver.close()

    

Wall time: 19min 58s
Wall time: 10min 30s


## Conectar na API do virasana para fazer UPLOAD do arquivo JSON resultante

O arquivo seguirá o formato [{_id: {Documento DUE}}]

In [33]:
print(len(conteineres_listadue))
print(len(due_detalhe))

1059
203


In [34]:
for conteiner, dues in list(conteineres_listadue.items())[:40]:
    if dues is not None and len(dues) > 0:
        print(conteiner, dues)
        
for due, conteudo in list(due_detalhe.items())[:10]:
    print(due, json.dumps(conteudo)[:30])

MNBU9083170 ['19BR0015082724']
SUDU1185280 ['19BR0014926309']
MCRU2007420 ['19BR0014920378']
MCRU2032150 ['19BR0014863528']
MCRU2013572 ['19BR0014917920']
MCRU2069078 ['19BR0014986301']
19BR0015082724 {"canal": "VERDE", "canalBD": 
19BR0014926309 {"canal": "VERDE", "canalBD": 
19BR0014920378 {"canal": "VERDE", "canalBD": 
19BR0014863528 {"canal": "VERDE", "canalBD": 
19BR0014917920 {"canal": "VERDE", "canalBD": 
19BR0014986301 {"canal": "VERDE", "canalBD": 
19BR0015036544 {"canal": "VERDE", "canalBD": 
19BR0015017728 {"canal": "VERDE", "canalBD": 
19BR0015037222 {"canal": "VERDE", "canalBD": 
19BR0015171611 {"canal": "VERDE", "canalBD": 


In [35]:
with open('dues.txt', 'w') as out:
    json.dump(due_detalhe, out)

In [36]:
with open('dues.txt', 'r') as due_in:
    dues_ = json.load(due_in)

In [38]:
dues_['19BR0015082724']

{'canal': 'VERDE',
 'canalBD': 1,
 'chaveAcesso': '19SBA025569377',
 'dataProcessamentoTa': '2019-11-04T06:44:58.000+0000',
 'exportadorEstrangeiro': False,
 'declarantePJBD': '01996609000125',
 'descricaoResultadoProcessarTA': 'Pendência não impeditiva do embarque',
 'despachoEmRecintoAlfandegado': True,
 'embarqueEmRecintoAlfandegado': True,
 'despachoEmRecintoDomiciliar': False,
 'formaExportacao': 'Por conta própria',
 'formaExportacaoBD': 1,
 'id': 2587996,
 'indicadorDAT': False,
 'indicadorExigencia': False,
 'indicadorImpedeEmbarqueTA': 'N',
 'indicadorInspecao': 'N',
 'indicadorOEA': False,
 'informacoesComplementares': 'JOÃO FLAVIO DE CARVALHO - CPF: 133.732.358-63 - REG: 8D.03.379\nREGINALDO JOÃO DE CARVALHO - CPF:247.658.238-73\nFATURA:279/2019',
 'usuarioConfaz': False,
 'consultaConfaz': False,
 'listaExigencias': [],
 'listaHistorico': [{'dataHoraEvento': '01/11/2019 - 14:26:04',
   'evento': 'Registro',
   'responsavel': '13373235863',
   'tipoEvento': 'REGISTRADA'},
  

In [42]:
def monta_due_ajna(due):
    def get_dados_recinto(recinto_dict):
        result = {}
        try:
            result['codigo'] = recinto_dict.get('codigo')
            depositario = recinto_dict.get('depositario')
            if depositario:
                result['depositario'] = depositario.get('depositario')
                result['nome'] = depositario.get('nome')
                result['descricao'] = depositario.get('descricao')
                unidade = depositario.get('unidadeLocalRFB')
                if unidade:
                    unidadeLocalRFB = depositario.get('codigo')
        except AttributeError:
            return None

        return result 
    
    keys = ['canal', 'chaveAcesso', 'dataProcessamentoTa', 'descricaoTipoItemDue',
            'exportadorEstrangeiro', 'formaExportacao', 'indicadorOEA', 'informacoesComplementares']
    pacote = {}
    for key in keys:
        item = due.get(key)
        if item is not None:
            pacote[key] = item

    declarante = due.get('niDeclarante')
    if declarante:
        pacote['Declarante'] = declarante.get('numero')
        pacote['Nome Declarante'] = declarante.get('nome')

    destino = due.get('paisImportador')
    if destino:
        pacote['PaisImportador'] = destino.get('nome')

    lista_items = due.get('listaInfoItemDue')
    itensDue = []
    for item in lista_items:
        itemDue = {}
        itemDue['descricaoMercadoria'] = item.get('descricaoMercadoria')
        itemDue['exportadorEstrangeiro'] = item.get('exportadorEstrangeiro')
        ncm = item.get('ncm')
        if ncm:
            itemDue['ncm'] = ncm.get('codigo')
        exportador = item.get('niExportador')
        if exportador:
            itemDue['Exportador'] = exportador.get('numero')
            itemDue['NomeExportador'] = exportador.get('nome')
        itensDue.append(itemDue)
    pacote['itens'] = itensDue
    
    for recinto_tipo in ['recintoAduaneiroDespacho',  'recintoAduaneiroEmbarque']:
        pacote[recinto_tipo] = get_dados_recinto(due.get(recinto_tipo))
    ruc = due.get('ruc')
    if ruc:
        pacote['ruc'] = ruc.get('numero')
    pacote['listaHistorico'] = due.get('listaHistorico')

    return pacote

    


In [43]:
pacote = monta_due_ajna(dues_['19BR0015082724'])
pacote

{'canal': 'VERDE',
 'chaveAcesso': '19SBA025569377',
 'dataProcessamentoTa': '2019-11-04T06:44:58.000+0000',
 'descricaoTipoItemDue': ' Nota Fiscal Eletrônica (NF-e)',
 'exportadorEstrangeiro': False,
 'formaExportacao': 'Por conta própria',
 'indicadorOEA': False,
 'informacoesComplementares': 'JOÃO FLAVIO DE CARVALHO - CPF: 133.732.358-63 - REG: 8D.03.379\nREGINALDO JOÃO DE CARVALHO - CPF:247.658.238-73\nFATURA:279/2019',
 'Declarante': '01996609000125',
 'Nome Declarante': 'SUPRACITRUS COMERCIAL LTDA.                                                                                                                           ',
 'PaisImportador': 'PAISES BAIXOS (HOLANDA)',
 'itens': [{'descricaoMercadoria': 'LIMAO TAHITI IN-NATURA,ACONDICIONADO EM EMBALAGEM GRAFICA PERSONALIZADA(PESO LIQUIDO 4,5KG DE LIMAO TAHITI IN- NATURA)',
   'exportadorEstrangeiro': False,
   'ncm': '8055000',
   'Exportador': '01996609000125',
   'NomeExportador': 'SUPRACITRUS COMERCIAL LTDA.                      

In [44]:
pacote_carregamento = {}
for conteiner, numeros_dues in conteineres_listadue.items():
    _id = conteineres_ids[conteiner]
    lista_dues = []
    for due in numeros_dues:
        if due is not None:
            pacote = monta_due_ajna(due_detalhe[due])
            lista_dues.append({'numero': due, **pacote})
    if numeros_dues and len(numeros_dues) > 0:
        pacote_carregamento[_id] = lista_dues
    

In [24]:
pacote_carregamento

{'5dc03a5936aeb574867c30dd': [{'numero': '19BR0015099473',
   'canal': 'VERDE',
   'chaveAcesso': '19QPQ025586360',
   'dataProcessamentoTa': '2019-11-03T01:30:21.000+0000',
   'descricaoTipoItemDue': ' Nota Fiscal Eletrônica (NF-e)',
   'exportadorEstrangeiro': False,
   'formaExportacao': 'Por conta própria',
   'indicadorOEA': False,
   'Declarante': '71352553000151',
   'Nome Declarante': 'EXPOCACCER- COOPERATIVA DOS CAFEICULTORES DO CERRADO LTDA.',
   'PaisImportador': 'ESTADOS UNIDOS',
   'itens': [{'descricaoMercadoria': 'CAFE CRU, N/D, EM GRAO, ARABICA, INFERIOR A COB 6, SEM DESCRICAO DE PENEIRAS, BEBIDA DURA OU DURA-RIADA.',
     'exportadorEstrangeiro': False,
     'ncm': '9011110',
     'Exportador': '71352553000151',
     'NomeExportador': 'EXPOCACCER- COOPERATIVA DOS CAFEICULTORES DO CERRADO LTDA.'},
    {'descricaoMercadoria': 'CAFE CRU, N/D, EM GRAO, ARABICA, INFERIOR A COB 6, SEM DESCRICAO DE PENEIRAS, BEBIDA DURA OU DURA-RIADA.',
     'exportadorEstrangeiro': False,
  

In [45]:
r = requests.post(VIRASANA_URL + "dues/update", json=pacote_carregamento, verify=False)
print(r.status_code)
print(r.text)


c:\users\25052288840\pycharmprojects\ajna_docs\virasana\venv\lib\site-packages\urllib3\connectionpool.py:851: InsecureRequestWarning:

Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings



201
{"status":"DUEs inseridas/atualizadas"}

